In [4]:
# ===============================
# Step 1: Install Dependencies
# ===============================
!pip install -q langchain langchain-openai langchain-groq langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00


In [6]:
# ===============================
# Step 2: Import Libraries
# ===============================
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory # Updated import
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain.memory.chat_message_histories import FileChatMessageHistory
from IPython.display import display, Markdown
from google.colab import userdata
import os

In [7]:
# ===============================
# Step 3: Setup LLM
# ===============================
groq_api_key = userdata.get("GROQ_API_KEY")  # or hardcode
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    api_key=groq_api_key,
    temperature=0.3,
)

In [8]:
# ===============================
# Step 4: Setup MemorySaver Logic
# ===============================
# Custom MemorySaver using file system storage
# The LocalFileStore is not available in the current version of langchain-community
# store = LocalFileStore("./chat_memory/")

# def get_session_history(session_id: str):
#     return FileChatMessageHistory(session_id, store)

# Using ConversationBufferMemory for simplicity
memory = ConversationBufferMemory(return_messages=True)

<ipython-input-8-434189519>:12: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages=True)


In [9]:
# ===============================
# Step 5: Prompt with Placeholder
# ===============================
chat_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("You are a helpful assistant that remembers previous conversations."),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{input}"),
])

In [12]:
# ===============================
# Step 6: Create RunnableWithMessageHistory
# ===============================
chat_chain = chat_prompt | llm

# Updated to use ConversationBufferMemory directly
chatbot = RunnableWithMessageHistory(
    chat_chain,
    lambda session_id: memory.chat_memory, # Pass the internal chat history
    input_messages_key="input",
    history_messages_key="history",
)

In [14]:
# ===============================
# 💬 Step 7: Chat with Memory Persistence
# ===============================
session_id = "memory_saver_session_001"

user_inputs = [
    "Hi! My name is GOTAN.",
    "Can you remind me what I just told you?",
    "What's the capital of France?",
    "And what's my name?"
]

for input_text in user_inputs:
    response = chatbot.invoke(
        {"input": input_text},
        config={"configurable": {"session_id": session_id}},
    )
    display(Markdown(f"### ❓ User: {input_text}"))
    display(Markdown(f"**🤖 Assistant:** {response.content}"))

### ❓ User: Hi! My name is GOTAN.

**🤖 Assistant:** Wait, I think there's been a change. Earlier, you told me your name was Shreyash, but now you're saying it's GOTAN. I'll update my memory to reflect the new information. So, to confirm, you'd like me to address you as GOTAN from now on, right? (Just a note: I'll still remember that you initially introduced yourself as Shreyash, but I'll prioritize the new name you've given me.)

### ❓ User: Can you remind me what I just told you?

**🤖 Assistant:** You just told me that your name is GOTAN. However, I should mention that this is a change from what you initially told me, which was that your name is Shreyash.

### ❓ User: What's the capital of France?

**🤖 Assistant:** The capital of France is Paris. And, as I recall, your name is currently GOTAN (although I still remember that you initially introduced yourself as Shreyash).

### ❓ User: And what's my name?

**🤖 Assistant:** Your name is GOTAN. I've been keeping track of our conversation, and although you initially told me your name was Shreyash, you later updated it to GOTAN, which is what I'll use to address you.